In [1]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

# 导入依赖
import warnings

warnings.filterwarnings("ignore")

# Disable logging messages unless there is an error
set_log_level("ERROR")

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [2]:
data_location = "/home/ann/opt/iNeuralProphet/data/"
df = pd.read_csv(data_location + "air_passengers.csv")
df.tail(3)

,ds,y
141,1960-10-01,461
142,1960-11-01,390
143,1960-12-01,432


In [3]:
m = NeuralProphet(n_lags=5, n_forecasts=3)
metrics_train = m.fit(df=df, freq="MS")

Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/209 [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

In [4]:
forecast = m.predict(df)
forecast.to_csv('forecast.csv')

Predicting: |          | 0/? [00:00<?, ?it/s]

最新预测的 df 进行数据分析

In [5]:
df_fc = m.get_latest_forecast(forecast)
df_fc.head(3)

,ds,y,origin-0
0,1960-10-01,461,473.350037
1,1960-11-01,390,423.013031
2,1960-12-01,432,470.072205


包括最新预报之前的 5 个步骤

In [6]:
df_fc = m.get_latest_forecast(forecast, include_previous_forecasts=5)
df_fc.head(10)

,ds,y,origin-5,origin-4,origin-3,origin-2,origin-1,origin-0
0,1960-05-01,472,452.847687,None,None,None,None,None
1,1960-06-01,535,519.563477,527.075989,None,None,None,None
2,1960-07-01,622,549.820923,554.184448,570.217285,None,None,None
3,1960-08-01,606,None,542.661865,562.529785,596.637695,None,None
4,1960-09-01,508,None,None,487.217499,547.181763,554.877197,None
5,1960-10-01,461,None,None,None,496.414948,490.471344,473.350037
6,1960-11-01,390,None,None,None,None,446.316711,423.013031
7,1960-12-01,432,None,None,None,None,None,470.072205


包含历史数据

In [7]:
df_fc = m.get_latest_forecast(forecast, include_history_data=True)
df_fc.head(100)

,ds,y,origin-0
0,1949-01-01,112,None
1,1949-02-01,118,None
2,1949-03-01,132,None
3,1949-04-01,129,None
4,1949-05-01,121,None
...,...,...,...
95,1956-12-01,306,None
96,1957-01-01,315,None
97,1957-02-01,301,None
98,1957-03-01,356,None


调用 predict raw=True，我们得到一个 df，其中每个 “step<i>” 指的是从该行的日期时间开始的 <i> 第 个提前一步预测。这里，<i> 是指预测的目标提前多少步.
例如，step0 是对此日期时间的预测。step1 是对下一个日期时间的预测

In [8]:
df = pd.read_csv(data_location + "air_passengers.csv")
forecast = m.predict(df, decompose=False, raw=True)
forecast.tail(3)

Predicting: |          | 0/? [00:00<?, ?it/s]

,ds,step0,step1,step2
134,1960-08-01,596.637695,547.181763,496.414948
135,1960-09-01,554.877197,490.471344,446.316711
136,1960-10-01,473.350037,423.013031,470.072205


设置 decompose=True 将包括各个预测组件，这些组件还引用它们各自对未来 step<i> 的贡献

In [9]:
df = pd.read_csv(data_location + "air_passengers.csv")
forecast = m.predict(df, decompose=True, raw=True)
forecast.tail(3)

Predicting: |          | 0/? [00:00<?, ?it/s]

,ds,step0,trend0,season_yearly0,ar0,step1,trend1,season_yearly1,ar1,step2,trend2,season_yearly2,ar2
134,1960-08-01,596.637695,480.693451,76.682625,39.261631,547.181763,483.188873,15.855521,48.137341,496.414948,485.603729,-24.855297,35.666489
135,1960-09-01,554.877197,483.188873,15.855521,55.832771,490.471344,485.603729,-24.855297,29.722874,446.316711,488.099121,-64.326599,22.544188
136,1960-10-01,473.350037,485.603729,-24.855297,12.601562,423.013031,488.099121,-64.326599,-0.759486,470.072205,490.514038,-32.291817,11.850000


将预测扩展到未知未来的方法

先生成预测的数据框

In [10]:
df = pd.read_csv(data_location + "air_passengers.csv")
future = m.make_future_dataframe(
    df, periods=3, n_historic_predictions=False
)  # periods=m.n_forecasts, n_historic_predictions=False

In [11]:
future.tail(10)

,ds,y
0,1960-08-01,606
1,1960-09-01,508
2,1960-10-01,461
3,1960-11-01,390
4,1960-12-01,432
5,1961-01-01,None
6,1961-02-01,None
7,1961-03-01,None


基于预测目标的数据框执行预测

In [12]:
forecast = m.predict(future)
forecast.tail(3)

Predicting: |          | 0/? [00:00<?, ?it/s]

,ds,y,yhat1,yhat2,yhat3,ar1,ar2,ar3,trend,season_yearly
5,1961-01-01,NaN,441.19281,NaN,NaN,-22.236631,NaN,NaN,493.009399,-29.579958
6,1961-02-01,NaN,NaN,448.442749,NaN,NaN,-8.513306,NaN,495.504822,-38.548756
7,1961-03-01,NaN,NaN,NaN,482.432434,NaN,NaN,-10.649454,497.758759,-4.676874


​raw 和 decompose参数控制输出内容
​raw=False, decompose=True(默认)​​ ​最常用。每个时间点一行，并显示成分分解。大多数情况下的分析和可视化。

raw=True, decompose=False 保留预测序列的原始结构，但隐藏成分细节。高级模型诊断，滚动预测分析。

raw=True, decompose=True ​最详细。保留原始结构且显示成分分解。深度调试，需要完全理解模型如何生成每一个预测。

raw=False, decompose=False ​最简洁。只返回最终预测值，没有分解细节。只需要最终预测结果用于下游系统集成。


In [13]:
forecast = m.predict(future, raw=False, decompose=True)
forecast

Predicting: |          | 0/? [00:00<?, ?it/s]

,ds,y,yhat1,yhat2,yhat3,ar1,ar2,ar3,trend,season_yearly
0,1960-08-01,606.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1960-09-01,508.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1960-10-01,461.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1960-11-01,390.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1960-12-01,432.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1961-01-01,NaN,441.19281,NaN,NaN,-22.236631,NaN,NaN,493.009399,-29.579958
6,1961-02-01,NaN,NaN,448.442749,NaN,NaN,-8.513306,NaN,495.504822,-38.548756
7,1961-03-01,NaN,NaN,NaN,482.432434,NaN,NaN,-10.649454,497.758759,-4.676874


In [14]:
forecast = m.predict(future, raw=True, decompose=False)
forecast

Predicting: |          | 0/? [00:00<?, ?it/s]

,ds,step0,step1,step2
0,1961-01-01,441.19281,448.442749,482.432434


In [15]:
forecast = m.predict(future, raw=True, decompose=True)
forecast

Predicting: |          | 0/? [00:00<?, ?it/s]

,ds,step0,trend0,season_yearly0,ar0,step1,trend1,season_yearly1,ar1,step2,trend2,season_yearly2,ar2
0,1961-01-01,441.19281,493.009399,-29.579958,-22.236631,448.442749,495.504822,-38.548756,-8.513306,482.432434,497.758759,-4.676874,-10.649454


In [16]:
forecast = m.predict(future, raw=False, decompose=False)
forecast

Predicting: |          | 0/? [00:00<?, ?it/s]

,ds,y,yhat1,yhat2,yhat3
0,1960-08-01,606.0,NaN,NaN,NaN
1,1960-09-01,508.0,NaN,NaN,NaN
2,1960-10-01,461.0,NaN,NaN,NaN
3,1960-11-01,390.0,NaN,NaN,NaN
4,1960-12-01,432.0,NaN,NaN,NaN
5,1961-01-01,NaN,441.19281,NaN,NaN
6,1961-02-01,NaN,NaN,448.442749,NaN
7,1961-03-01,NaN,NaN,NaN,482.432434
